In [1]:
# load libraries
import xarray as xr
import numpy as np

from argopy import DataFetcher as ArgoDataFetcher

from datetime import datetime, timedelta
import pandas as pd

# User defined functions:
def get_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax,time_in,time_f):
    
    """Function to get argo data for a given lat,lon box (using Argopy), 
       and return a 2D array collection of vertical profile for the given region
    
    Parameters
    ----------
    llon : int
        left longitude
    rlon : int
        right longtidue
    ulat : int
        upper latitude
    llat : int
        lower latitude
    time_in : str/datetime object
        the start time of desired range, formatted Y-m-d
    time_f : str/datetime object
        the end time of desired range, formatted Y-m-d
    
    
    Returns
    ---------
    xarray
        The result is a xarray of the vertical profile for the given range and region.
        
    
    """

    ds_points = ArgoDataFetcher(src='erddap').region([llon,rlon, llat,ulat, depthmin, depthmax,time_in,time_f]).to_xarray()
    ds_profiles = ds_points.argo.point2profile()
    return ds_profiles

def spliced_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax,time_start,time_end):
    """Function that gets the argo data for given latitude and longitude bounding box
       (using Argopy), and given start and end time range to return a 2D array collection of vertical
       profile for the given region and time frame
    
    Parameters
    ----------
    llon : int
        left longitude
    rlon : int
        right longtidue
    ulat : int
        upper latitude
    llat : int
        lower latitude
    time_in : str/datetime object
        the start time of desired range, formatted Y-m-d
    time_f : str/datetime object
        the end time of desired range, formatted Y-m-d
    
    
    Returns
    ---------
    xarray
        The result is a xarray of the vertical profile for the given range and region.
        
    
    """

    
    #step
    max_dt = timedelta(days = 10)
    
    if isinstance(time_start, str):
        time_start = datetime.strptime(time_start,"%Y-%m-%d")
    if isinstance(time_end, str):
        time_end = datetime.strptime(time_end,"%Y-%m-%d")
        
    if time_end - time_start <= max_dt:
        ds = get_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax,time_start,time_end)
        return ds
    else:
        early_end = time_start+max_dt
        ds = get_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax,time_start,early_end)
        print("Retrived data from " + str(time_start) + " to " + str(early_end) + ", retreived " + str(len(ds.N_PROF)) + " profiles")
        ds2 = spliced_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax, early_end,time_end)
        return xr.concat([ds,ds2],dim='N_PROF')


In [6]:
llon=-90;rlon=0
ulat=70;llat=0 
depthmin=0;depthmax=1400
time_start='2014-01-01'
time_end='2014-03-01'
ds=spliced_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax,time_start,time_end)

Retrived data from 2014-01-01 00:00:00 to 2014-01-11 00:00:00, retreived 525 profiles
Retrived data from 2014-01-11 00:00:00 to 2014-01-21 00:00:00, retreived 532 profiles
Retrived data from 2014-01-21 00:00:00 to 2014-01-31 00:00:00, retreived 517 profiles
Retrived data from 2014-01-31 00:00:00 to 2014-02-10 00:00:00, retreived 533 profiles
Retrived data from 2014-02-10 00:00:00 to 2014-02-20 00:00:00, retreived 552 profiles


In [7]:
year=2014
ds.to_netcdf(str('/home/jovyan/ohw20-proj-pyxpcm/data/winter'+str(year)+'.nc'))

In [2]:
llon=-90;rlon=0
ulat=70;llat=0 
depthmin=0;depthmax=1400
time_start='2014-07-01'
time_end='2014-09-01'
ds=spliced_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax,time_start,time_end)

Retrived data from 2014-07-01 00:00:00 to 2014-07-11 00:00:00, retreived 587 profiles
Retrived data from 2014-07-11 00:00:00 to 2014-07-21 00:00:00, retreived 615 profiles
Retrived data from 2014-07-21 00:00:00 to 2014-07-31 00:00:00, retreived 581 profiles
Retrived data from 2014-07-31 00:00:00 to 2014-08-10 00:00:00, retreived 570 profiles
Retrived data from 2014-08-10 00:00:00 to 2014-08-20 00:00:00, retreived 580 profiles
Retrived data from 2014-08-20 00:00:00 to 2014-08-30 00:00:00, retreived 562 profiles


In [3]:
year=2014
ds.to_netcdf(str('/home/jovyan/ohw20-proj-pyxpcm/data/summer'+str(year)+'.nc'))